# Spanning trees

Anton Antonov   
July 2024

----

## Setup

In [ ]:
use Graph;

use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Star;
use Graph::Path;
use Graph::Petersen;
use Graph::Wheel;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;
use Data::TypeSystem;
use Data::Translators;
use Data::Geographics;
use Math::DistanceFunctions;
use Math::Nearest;
use Hash::Merge;

use JavaScript::D3;
use WWW::MermaidInk;

In [ ]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [ ]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

In [ ]:
my $title-color = 'Silver';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

my %opts = :$background, :$title-color, :$edge-thickness, :$vertex-size;

-----

## Small Geo-points set

A company is planning a fiber network for a number of Chicago suburbs. It only has the right of way for its fiber along certain corridors. Some of those corridors might be more expensive. Find the subgraph of connection corridors that connect every suburb with the lowest total cost:

In [ ]:
my @dsEdges =
        %('from'=>'Maywood','to'=>'Downers','weight'=>14),
        %('from'=>'Maywood','to'=>'Oakbrook','weight'=>28),
        %('from'=>'Maywood','to'=>'Addison','weight'=>10),
        %('from'=>'Downers','to'=>'Grove','weight'=>25),
        %('from'=>'Downers','to'=>'Stickney','weight'=>23),
        %('from'=>'Downers','to'=>'Wheaton','weight'=>20),
        %('from'=>'Downers','to'=>'Oakbrook','weight'=>27),
        %('from'=>'Oakbrook','to'=>'Stickney','weight'=>11),
        %('from'=>'Oakbrook','to'=>'Addison','weight'=>17),
        %('from'=>'Addison','to'=>'McCook','weight'=>21),
        %('from'=>'Grove','to'=>'Wheaton','weight'=>10),
        %('from'=>'Stickney','to'=>'Wheaton','weight'=>15),
        %('from'=>'Stickney','to'=>'Bellwood','weight'=>20),
        %('from'=>'Wheaton','to'=>'Bellwood','weight'=>10),
        %('from'=>'Bellwood','to'=>'McCook','weight'=>28);

deduce-type(@dsEdges)

In [ ]:
my $g = Graph.new(@dsEdges)

In [ ]:
#% js
$g.edges(:dataset)
==> js-d3-graph-plot(
    width => 800,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => { strength => -800}, collision => {radius => 50}}
    )

In [ ]:
my $stree = $g.find-spanning-tree;

In [ ]:
#% js
$g.edges
==> js-d3-graph-plot(
    highlight => $stree.edges,
    width => 800,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => { strength => -800}, collision => {radius => 50}}
    )

-------

## Grid graph

In [ ]:
my $g = Graph::Grid.new(n => 4, m => 6)

In [ ]:
my $stree = $g.find-spanning-tree

In [ ]:
#% js
$g.edges
==> js-d3-graph-plot(
    highlight => [|$stree.vertex-list, |$stree.edges],
    width => 500,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => {strength => -160, iterations => 2}, collision => {radius => 4, iterations => 1}, link => {distance => 0}}
    )

-----

## Florida cities

Build an interstate highway system joining the cities of the state of Florida, USA.

Here is a dataset of large enough cities:

In [ ]:
my @cities = city-data().grep({ $_<State> eq 'Florida' && $_<Population> ≥ 50_000 });
@cities.elems

In [ ]:
#% html
@cities.head(4) ==> to-html

Here is the corresponding weighted graph:

In [ ]:
my %coords = @cities.map({ $_<City> => ($_<Latitude>, $_<Longitude>) });
my @dsEdges = (%coords.keys X %coords.keys ).map({ %(from => $_.head, to => $_.tail, weight => geo-distance(|%coords{$_.head}, |%coords{$_.tail} )) })

In [ ]:
my $g = Graph.new(@dsEdges, :!directed)

In [ ]:
my $stree = $g.find-spanning-tree(method => 'kruskal')

In [ ]:
my $stree2 = $g.find-spanning-tree(method => 'prim')

In [ ]:
#% js
$stree2.edges
==> js-d3-graph-plot(
    width => 1200,
    height => 1000,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => {strength => -300, iterations => 2}, collision => {radius => 4, iterations => 1}, link => {distance => 0}}
    )